In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from ansys.mapdl.core import launch_mapdl
from ansys.mapdl import reader as pymapdl_reader

In [2]:
# Start an MAPDL instance
mapdl = launch_mapdl()

In [3]:
mapdl.finish()
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   02:32:07  NOV 20, 2024 CP=      0.172

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

## Triangular Lattice

In [4]:
design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
total_z = 5/1000
# density = 0.5

wall_thickness = 0.5/1000
n_cell = 5
external_wall = False
external_wall_thickness = wall_thickness

In [5]:
cell_size = (design_height - 2*5/6*np.sqrt(3)*wall_thickness
              - (n_cell - 1)*np.sqrt(3)*wall_thickness)/n_cell + np.sqrt(3)*wall_thickness

In [6]:
i = 0
design_part = mapdl.rectng(0, design_width, 0, design_height)
while True:
    if i%2 == 0:
        for j in range(n_cell):
            k1 = mapdl.k(x = i*cell_size*np.sqrt(3)/2, y = j*cell_size + 5/6*np.sqrt(3)*wall_thickness)
            k2 = mapdl.k(x = i*cell_size*np.sqrt(3)/2, y = (j+1)*cell_size - 1/6*np.sqrt(3)*wall_thickness)
            k3 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - 3/2*wall_thickness, y = (j+1/2)*cell_size + 1/3*np.sqrt(3)*wall_thickness)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                pass
            
        for j in range(n_cell-1):
            k1 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness, y = (j+1/2)*cell_size + 5/6*np.sqrt(3)*wall_thickness)
            k2 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness, y = (j+3/2)*cell_size - 1/6*np.sqrt(3)*wall_thickness)
            k3 = mapdl.k(x = i*cell_size*np.sqrt(3)/2 + 1/2*wall_thickness, y = (j+1)*cell_size + 1/3*np.sqrt(3)*wall_thickness)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                pass
        k1 = mapdl.k(x = i*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = 0)
        k2 = mapdl.k(x = (i+2)*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = 0)
        k3 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = 1/2*cell_size)
        hole = mapdl.a(k1, k2, k3)
        try:
            design_part = mapdl.asba(na1 = design_part, na2 = hole)
        except:
            pass
        k1 = mapdl.k(x = i*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = design_height)
        k2 = mapdl.k(x = (i+2)*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = design_height)
        k3 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = design_height - 1/2*cell_size)
        hole = mapdl.a(k1, k2, k3)
        try:
            design_part = mapdl.asba(na1 = design_part, na2 = hole)
        except:
            pass
    else:
        for j in range(n_cell):
            k1 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness, y = j*cell_size + 5/6*np.sqrt(3)*wall_thickness)
            k2 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness, y = (j+1)*cell_size - 1/6*np.sqrt(3)*wall_thickness)
            k3 = mapdl.k(x = i*cell_size*np.sqrt(3)/2 + 1/2*wall_thickness, y = (j+1/2)*cell_size + 1/3*np.sqrt(3)*wall_thickness)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                pass
        for j in range(n_cell-1):
            k1 = mapdl.k(x = i*cell_size*np.sqrt(3)/2, y = (j+1/2)*cell_size + 5/6*np.sqrt(3)*wall_thickness)
            k2 = mapdl.k(x = i*cell_size*np.sqrt(3)/2, y = (j+3/2)*cell_size - 1/6*np.sqrt(3)*wall_thickness)
            k3 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - 3/2*wall_thickness, y = (j+1)*cell_size + 1/3*np.sqrt(3)*wall_thickness)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                pass
    if (i+1)*cell_size*np.sqrt(3)/2 > design_width:
        break
    i+=1

In [7]:
mapdl.rectng(x1=-total_width/2+design_width/2, x2=0, y1=0, y2=design_height)
mapdl.rectng(design_width, total_width/2+design_width/2, 0, design_height)
if external_wall:
    mapdl.rectng(x1=0, x2=design_width, y1=0, y2=external_wall_thickness)
    mapdl.rectng(x1=0, x2=design_width, y1=design_height, y2=design_height-external_wall_thickness)
mapdl.aadd("all")

4

In [8]:
mapdl.vext("ALL", dz = total_z)

EXTRUDE ALL AREAS 
      IN DIRECTION   0.000000    ,  0.000000    , 0.5000000E-02
      SCALED         0.000000    ,  0.000000    ,  0.000000

In [9]:
mapdl.vplot(cpos="xy", show_lines=True)

Widget(value='<iframe src="http://localhost:59749/index.html?ui=P_0x281163558a0_0&reconnect=auto" class="pyvis…

In [11]:
mapdl.get(entity="VOLU", entnum=1, item1="VOLU")*1e9/(25*5*144)

0.7108913388888889

In [12]:
ex = 30e6  # Young's Modulus
prxy = 0.27  # Poisson's ratio

mapdl.mp("EX", 1, ex)
mapdl.mp("PRXY", 1, prxy)

MATERIAL          1     PRXY =  0.2700000

In [13]:
mapdl.allsel()

SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW

In [15]:
# Element type with plasticity capability (SOLID185 is suitable)
mapdl.et(1, 'SOLID187')

# Define mesh size
mapdl.esize(0.002)  # Smaller element size for better accuracy

# Mesh the volume
mapdl.vmesh('ALL')

GENERATE NODES AND ELEMENTS   IN  ALL  SELECTED VOLUMES  

 NUMBER OF VOLUMES MESHED   =         1
 MAXIMUM NODE NUMBER        =     52129
 MAXIMUM ELEMENT NUMBER     =     27672

 ------------------------------------------------------------------------------
            <<<<<<          SHAPE TESTING SUMMARY           >>>>>>
            <<<<<<       FOR NEW OR MODIFIED ELEMENTS       >>>>>>
 ------------------------------------------------------------------------------
                    --------------------------------------
                    |  Element count     27672 SOLID187  |
                    --------------------------------------

  Test                Number tested  Warning count  Error count    Warn+Err %
  ----                -------------  -------------  -----------    ----------
  Aspect Ratio              27672              8             0         0.03 %
  Maximum Angle             27672             11             0         0.04 %
  Jacobian Ratio            27672   

In [16]:
mapdl.eplot()

Widget(value='<iframe src="http://localhost:59749/index.html?ui=P_0x281182a4fa0_2&reconnect=auto" class="pyvis…